In [1]:
import pandas as pd
import requests
import bs4
GENRE_TYPES = 'Talk-Show|News'
# IMDB_PROFESSION_TYPES = 'actor|actress|miscellaneous|Null'

##Get nconst by using anchor name and pull year

In [2]:
def get_nconst(Anchor_Name,Pull_Year):

  Read_Titlebasics = pd.read_table('/content/drive/My Drive/Datasets/title.basics.tsv/data.tsv', sep='\t')
  Read_Namebasics = pd.read_table('/content/drive/My Drive/Datasets/name.basics.tsv/data.tsv', sep='\t')

  Read_Namebasics.primaryProfession = Read_Namebasics.primaryProfession.fillna("Null")
  filter_year = Read_Namebasics[Read_Namebasics.deathYear >= Pull_Year]
  Search_name = filter_year[filter_year.primaryName == Anchor_Name]
  # filter_profession = Search_name[Search_name.primaryProfession.str.contains(IMDB_PROFESSION_TYPES)]
  # return filter_profession
  # return Search_name
  # return len(filter_profession.index)
  if len(Search_name.index) == 1:
    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
      return [row,prepare_list_kft]
  # else:

  #   query = Anchor_Name
  #   query = query.replace(' ', '+')
  #   url = f"https://google.com/search?q={query}"
  #   response = requests.get(url)
  #   response.raise_for_status()
  #   knowledge_panel_content = bs4.BeautifulSoup(response.text, 'html.parser')

  #   list_content = []
  #   for i in knowledge_panel_content.select('span'):
  #     list_content.append(i.getText())
  #   if "Born" in list_content:
  #     born_index = list_content.index("Born")
  #     dob = list_content[born_index+2]
  #     exact_year=dob.split(', ')[1].split(' (')[0]
  #     # print(exact_year)

  #     filter_dob = (Search_name[Search_name.birthYear ==exact_year])
  #     # print(filter_dob)
  #     # print(filter_dob.knownForTitles)
  #     for index, row in filter_dob.iterrows():
  #       prepare_list_kft = row.knownForTitles.split(',')
  #       return [row,prepare_list_kft]
  else:
    # return Search_name
    # qwe = Search_name[Search_name.knownForTitles]
    # return qwe

    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
    
      # print(prepare_list_kft)
      for i in prepare_list_kft:
        # print(i)
        # print(Read_Titlebasics)
        take_title = Read_Titlebasics[Read_Titlebasics.tconst == i]
        # print(take_title) 
        check_genres = take_title[take_title.genres.str.contains(GENRE_TYPES)]
        # print(check_genres)
        if len(check_genres.index) == 1:
          print(prepare_list_kft)
          print(check_genres)
          return [row,prepare_list_kft]

In [4]:
get_nconst("Jim Clancy","2006")[0]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['tt2788528', 'tt1160318', 'tt2364518', 'tt0827984']
            tconst  titleType  ... runtimeMinutes genres
3415890  tt2788528  tvSpecial  ...             \N   News

[1 rows x 9 columns]


nconst                                             nm2330770
primaryName                                       Jim Clancy
birthYear                                                 \N
deathYear                                                 \N
primaryProfession                            writer,producer
knownForTitles       tt2788528,tt1160318,tt2364518,tt0827984
Name: 3416014, dtype: object

In [15]:
get_nconst("Colleen Williams","2006")[1]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


['tt0256549', 'tt0382458', 'tt3444938']

In [14]:
get_nconst("Colleen Williams","2006")[0]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
2673061,nm1497536,Colleen Williams,1955,\N,Null,"tt0256549,tt0382458,tt3444938"


## Getting all shows

In [3]:
def get_all_shows(nConst):
  chunksize = 6*10**6
  titleprincipal = pd.read_table('/content/drive/My Drive/Datasets/title.principals.tsv.gz', chunksize = chunksize)
  all_shows = pd.DataFrame(columns=['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'])
  #Initializing with the same columns as title.principals
  for i,chunk in enumerate((titleprincipal)):
    req_shows = chunk[chunk['nconst'] == nConst]
    print(' {} shows in chunk {}'.format(len(req_shows), i))
    all_shows = all_shows.append(req_shows, ignore_index = True)
    # filter_category = all_shows[all_shows.category == 'self']

  return all_shows.tconst.tolist()

In [10]:
get_all_shows(get_nconst("Tony Harris","2006")[0].nconst)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


 0 shows in chunk 0
 0 shows in chunk 1
 0 shows in chunk 2
 0 shows in chunk 3
 0 shows in chunk 4
 1 shows in chunk 5
 0 shows in chunk 6


['tt7005704']

In [ ]:
Shows_list = list(set().union(get_nconst("Colleen Williams","2006")[1],get_all_shows(get_nconst("Colleen Williams","2006")[0].nconst)))
Shows_list

In [6]:
Shows_list =[] 
Combine_list = list(set().union(get_nconst("Colleen Williams","2006")[1],get_all_shows(get_nconst("Colleen Williams","2006")[0].nconst)))
titleakas = pd.read_table('/content/drive/My Drive/Datasets/title.akas.tsv/data.tsv', sep='\t')
# titleakas_US = titleakas[titleakas.region == 'US']
for title in Combine_list:  
  titleakas_titles = titleakas[titleakas.titleId == title]
  print(titleakas_titles)
  x = (titleakas_titles.region == 'US')
  print(x)
  if x is True:
    Shows_list.append(titleakas_titles.titleId)
    print(Shows_list)
  else:
    print('Not found')

KeyboardInterrupt: ignored

##Getting selected show by using filters

In [4]:
Combine_list = set().union(get_nconst("Matt Lauer","2006")[1],get_all_shows(get_nconst("Matt Lauer","2006")[0].nconst))

# All_shows = get_all_shows('nm8253813')
# tConsts = set(All_shows['tconst'])
tConsts= Combine_list

chunksize = 6*10**6

titlebasics = pd.read_table('/content/drive/My Drive/Datasets/title.basics.tsv/data.tsv', chunksize = chunksize)
Show_Names = pd.DataFrame(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
for i,chunk in enumerate(titlebasics):
  show_names = chunk[chunk['tconst'].isin(tConsts)]
  # show_names = chunk[chunk['tconst'] == 'tt2788528']
  print('{} shows in chunk {}'.format(len(show_names), i))
  Show_Names = Show_Names.append(show_names, ignore_index = True)
  # print(Show_Names)
  filter_title_type = Show_Names[Show_Names['titleType']=='tvSeries']
  filter_genres = filter_title_type[filter_title_type.genres.str.contains(GENRE_TYPES)]
  filter_end_year = filter_genres[filter_genres.endYear >= '2006']
  filter_start_year = filter_end_year[filter_end_year.startYear.apply(str) <='2006']
filter_start_year
# print(Primary_title_list)


# titleakas = pd.read_table('/content/drive/My Drive/Datasets/title.akas.tsv/data.tsv', sep='\t')
# titleakas.region = titleakas.region.fillna("US")
# Shows_list = []
# for i, title_name in enumerate(Primary_title_list):  
#   titleakas_titles = titleakas[titleakas.title == title_name]
#   # print(titleakas_titles)
#   x = (titleakas_titles.region == 'US')
#   # print(x)
#   if x is True:
#     Shows_list.append(title_name)
#     print(Shows_list)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


 43 shows in chunk 0
 70 shows in chunk 1
 128 shows in chunk 2
 118 shows in chunk 3
 385 shows in chunk 4
 258 shows in chunk 5
 3 shows in chunk 6


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1006 shows in chunk 0
3 shows in chunk 1


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0044298,tvSeries,Today,Today,0,1952,\N,240,"News,Talk-Show"
3,tt0261484,tvSeries,NewsChannel 4,NewsChannel 4,0,1995,\N,\N,News


In [5]:
Combine_list = set().union(get_nconst("Katie Couric","2006")[1],get_all_shows(get_nconst("Katie Couric","2006")[0].nconst))

# All_shows = get_all_shows('nm8253813')
# tConsts = set(All_shows['tconst'])
tConsts= Combine_list

chunksize = 6*10**6

titlebasics = pd.read_table('/content/drive/My Drive/Datasets/title.basics.tsv/data.tsv', chunksize = chunksize)
Show_Names = pd.DataFrame(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
for i,chunk in enumerate(titlebasics):
  show_names = chunk[chunk['tconst'].isin(tConsts)]
  # show_names = chunk[chunk['tconst'] == 'tt2788528']
  print('{} shows in chunk {}'.format(len(show_names), i))
  Show_Names = Show_Names.append(show_names, ignore_index = True)
  # print(Show_Names)
  filter_title_type = Show_Names[Show_Names['titleType']=='tvSeries']
  filter_genres = filter_title_type[filter_title_type.genres.str.contains(GENRE_TYPES)]
  filter_end_year = filter_genres[filter_genres.endYear >= '2006']
  filter_start_year = filter_end_year[filter_end_year.startYear.apply(str) <='2006']
filter_start_year

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


 67 shows in chunk 0
 46 shows in chunk 1
 99 shows in chunk 2
 363 shows in chunk 3
 137 shows in chunk 4
 47 shows in chunk 5
 13 shows in chunk 6


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


762 shows in chunk 0
13 shows in chunk 1


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
12,tt0433290,tvSeries,Fashion in Focus,Fashion in Focus,0,2003,\N,60,"Documentary,News"
67,tt0800281,tvSeries,CBS Evening News with Katie Couric,CBS Evening News with Katie Couric,0,2006,\N,25,News


In [25]:
filter_start_year

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0044298,tvSeries,Today,Today,0,1952,\N,240,"News,Talk-Show"
3,tt0261484,tvSeries,NewsChannel 4,NewsChannel 4,0,1995,\N,\N,News


In [9]:
Combine_list

{'tt0255325', 'tt0978075', 'tt7005704'}

In [29]:
Primary_title_list =  filter_start_year.primaryTitle.tolist()
Primary_title_list

['Anderson Cooper 360°',
 'American Morning',
 'CNN Newsroom',
 'CNN Saturday Morning']

In [30]:
af = open("/content/drive/My Drive/Datasets/Trial-1 S7-H11 video/2006-01-04_0000_US_00000063_V2_MB7_VHS7_H11_MS.txt3")
we = af.read()
for i in Primary_title_list:
  if we.find(i)!=-1:
    print("yes")
  else:
    print("No")


No
No
No
No
